# Combining with satellite data

## Load satellite data using odc-stac



In [1]:
import odc.stac
import pystac_client

# Connect to stac catalogue
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

# Set cloud defaults
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

# Build a query with the parameters above
bbox = [122.2186 - 0.05, -18.0008 - 0.05, 122.2186 + 0.05, -18.0008 + 0.05]
query = catalog.search(
    bbox=bbox,
    collections=["ga_ls8c_ard_3"],
    datetime="2024-01-01/2024-01-31",
)

# Search the STAC catalog for all items matching the query
ds = odc.stac.load(
    list(query.items()),
    bands=["nbart_red", "nbart_green", "nbart_blue"],
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
    fail_on_error=False,
    chunks={},
)

ds.isel(time=0).odc.explore(vmin=0, vmax=2000)

/env/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")
/env/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


## Using pixel_tides



In [2]:
from eo_tides.model import pixel_tides

tides_highres, tides_lowres = pixel_tides(
    ds=ds,
    model="FES2014",
    directory="../../tests/data/tide_models_tests"
)

tides_highres.isel(time=0).odc.explore()

Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using FES2014 in parallel


100%|██████████| 5/5 [00:01<00:00,  2.62it/s]


Reprojecting tides into original array
